# Requirements

In [4]:
import os
import json
import pandas as pd
import numpy as np
import json
import logging ### to monitor the code
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
from urllib.request import urlopen 
import io
import getpass
import re
from collections import defaultdict
from itertools import islice # to iterate through dicts

import nltk
from nltk.collocations import *

#!pip3 install pyconll #
import pyconll # universal dependencies parser

### plotting
### to use latex (important for greek fonts)
#! apt-get install texlive-latex-recommended 
#! apt install texlive-latex-extra
#! apt install dvipng
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

#from anda import gr
# OR uncomment the following:
script_url = "https://raw.githubusercontent.com/sdam-au/anda_py/master/anda/gr.py"
exec(requests.get(script_url).content)

In [6]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [8]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1iVta_FuEDgUM_Lf_yByrdbbXNoVH_dnVZs6QRyYv1NM/edit?usp=sharing")

# AGT_metadata = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hEUnL3E07F-EnE3wYnk1V91aXfPDrcnhFHKjD-04CM0/edit?usp=sharing")

In [9]:
# import the data
AGT = sddk.read_file("SDAM_data/AGT/AGT_dated_20201027.json", "df", conf)

In [10]:
AGT_string = " ".join(AGT["string"].tolist())
len(AGT_string)

219532769

In [11]:
test_string = "ἀρχόμενος ³σέο, Φοῖβε, text in latin also CAPITAL ⁴παλαιγενέων 1234 κλέα 34056 φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι."
test_string

'ἀρχόμενος ³σέο, Φοῖβε, text in latin also CAPITAL ⁴παλαιγενέων 1234 κλέα 34056 φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.'

In [12]:
# remove numerical superscripts
sups = ["¹", "²", "³", "⁴","⁵", "⁶", "⁷", "⁰", "⁹"]
def remove_sups(string):
    for sup in sups:
        string = string.replace(sup, "")
    return string

def grave_to_acute(string):
    GRAVE = "\u0300"
    ACUTE = "\u0301"
    return unicodedata.normalize("NFC", "".join(unicodedata.normalize("NFD", string).replace(GRAVE, ACUTE)))

test_string_1 = re.sub("[a-zA-Z0-9]", "", test_string)
test_string_1

'ἀρχόμενος ³σέο, Φοῖβε,      ⁴παλαιγενέων  κλέα  φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.'

In [13]:
test_string_2 = remove_sups(test_string_1)
test_string_3 = grave_to_acute(test_string_2)
print(test_string_3.split())

['ἀρχόμενος', 'σέο,', 'Φοῖβε,', 'παλαιγενέων', 'κλέα', 'φωτῶν', 'μνήσομαι,', 'οἵ', 'Πόντοιο', 'κατά', 'στόμα', 'καί', 'διά', 'πέτρας', 'Κυανέας', 'βασιλῆος', 'ἐφημοσύνῃ', 'Πελίαο', 'χρύσειον', 'μετά', 'κῶας', 'ἐύζυγον', 'ἤλασαν', 'Ἀργώ.', 'τοίην', 'γάρ', 'Πελίης', 'φάτιν', 'ἔκλυεν,', 'ὥς', 'μιν', 'ὀπίσσω', 'μοῖρα', 'μένει', 'στυγερή,', 'τοῦδʼ', 'ἀνέρος,', 'ὅντινʼ', 'ἴδοιτο', 'δημόθεν', 'οἰοπέδιλον,', 'ὑπʼ', 'ἐννεσίῃσι', 'δαμῆναι.']


In [14]:
# add column with sentences

def get_sentences(string):
  sentences = [s.strip() for s in re.split("\·|\.|\:|\;", unicodedata.normalize("NFC", string))]
  return sentences

AGT["sentences"] = AGT["string"].apply(get_sentences)

In [54]:
AGT.head(5)

filename              author  \
0  tlg0001.tlg001.perseus-grc2.xml  Apollonius Rhodius   
1  tlg0003.tlg001.perseus-grc2.xml          Thucydides   
2  tlg0004.tlg001.perseus-grc1.xml   Diogenes Laertius   
3  tlg0005.tlg001.perseus-grc1.xml          Theocritus   
4  tlg0005.tlg002.perseus-grc1.xml          Theocritus   

                           title  \
0                    Argonautica   
1          The Peloponnesian War   
2  Lives of Eminent Philosophers   
3                        Idylls    
4                      Epigrams    

                                              string  wordcount author_id  \
0  ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...      38822   tlg0001   
1  \nΘουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶ...     150126   tlg0003   
2  Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...     110773   tlg0004   
3  \n̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα...      19200   tlg0005   
4  τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...       1734   tlg0005   

           doc_id  raw_date  date_avr                  date_probs  \
0  tlg0001.tlg001    3 B.C.      -2.5                 {'-2.5': 1}   
1  tlg0003.tlg001    5 B.C.      -4.5                 {'-4.5': 1}   
2  tlg0004.tlg001    A.D. 3       2.5                  {'2.5': 1}   
3  tlg0005.tlg001  4-3 B.C.      -3.0  {'-3.5': 0.5, '-2.5': 0.5}   
4  tlg0005.tlg002  4-3 B.C.      -3.0  {'-3.5': 0.5, '-2.5': 0.5}   

   date_manual provenience                                          sentences  \
0         -2.5       pagan  [ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...   
1         -4.5       pagan  [Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...   
2          NaN              [Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...   
3          NaN              [̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...   
4          NaN              [τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...   

                                             lemmata  lemmata_wordcount  \
0  [Φοῖβος, παλαιγενής, κλέος, φάος, μιμνήσκω, πό...              23803   
1  [θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...              66680   
2  [φιλοσοφία, ἔργον, ἔνιοί, φάσις, βαρβαρόομαι, ...              53989   
3  [αδύ, ψιθύρισμα, πίτυς, αἰπόλος, ἐκεῖνος, πότη...              12000   
4  [ῥοδῆ, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλλο...               1071   

                                lemmatized_sentences  n_sentences  
0  [[Φοῖβος, παλαιγενής, κλέος, φάος, μιμνήσκω, π...         3233  
1  [[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...         6068  
2  [[φιλοσοφία, ἔργον, ἔνιοί, φάσις, βαρβαρόομαι,...        10244  
3  [[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, ἐκεῖνος, πότ...         1733  
4  [[ῥοδῆ, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλλ...          137

In [56]:
AGT.iloc[0]["string"][:500]

'ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι. δηρὸν δʼ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπʼ ἰλύος, ἄλλο δʼ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν. ἵκετο δʼ ἐς Πελίην α'

# Lemmatization

In [17]:
%%time
AGT["lemmata"] = AGT.apply(lambda row: lemmatize_string(row["string"], all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True), axis=1)

CPU times: user 2min 26s, sys: 2.2 s, total: 2min 28s
Wall time: 2min 29s


In [18]:
all_lemmata = []
for work in AGT["lemmata"].tolist():
    all_lemmata.extend(work)

In [21]:
AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183

13812934

In [22]:
%%time
def lemmatize_sentences(sentences):
    lemmatized_sentences = []
    for sentence in sentences:
        lemmatized_sentence = lemmatize_string(sentence, all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True)
        lemmatized_sentences.append(lemmatized_sentence)
    return lemmatized_sentences

AGT["lemmatized_sentences"] = AGT["sentences"].apply(lemmatize_sentences)

CPU times: user 2min 46s, sys: 403 ms, total: 2min 47s
Wall time: 2min 47s


In [23]:
AGT["n_sentences"] = AGT['lemmatized_sentences'].apply(lambda x: len(x))

In [24]:
# look at some string
# e.g. paul's letter to the Galatians
AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["string"][:1000]

'\n\n                    ΠΑΥΛΟΣ ἀπόστολος, οὐκ ἀπʼ ἀνθρώπων οὐδὲ διʼ ἀνθρώπου\n\nἀλλὰ διὰ Ἰησοῦ Χριστοῦ καὶ θεοῦ πατρὸς τοῦ\n\nἐγείραντος αὐτὸν ἐκ νεκρῶν,\n                     \n                    καὶ οἱ σὺν ἐμοὶ πάντες\n\nἀδελφοί, ταῖς ἐκκλησίαις τῆς Γαλατίας·\n                     \n                    χάρις ὑμῖν καὶ\n\n\n\n εἰρήνη ἀπὸ θεοῦ πατρὸς ἡμῶν καὶ κυρίου Ἰησοῦ Χριστοῦ,\n                     \n                    τοῦ δόντος ἑαυτὸν ὑπὲρ τῶν ἁμαρτιῶν ἡμῶν ὅπως ἐξέληται\n\nἡμᾶς ἐκ τοῦ αἰῶνος τοῦ ἐνεστῶτος πονηροῦ κατὰ τὸ\n\nθέλημα τοῦ θεοῦ καὶ πατρὸς ἡμῶν,\n                     \n                    ᾧ ἡ δόξα εἰς τοὺς\n\nαἰῶνας τῶν αἰώνων· ἀμήν.\n                     \n\n                    Θαυμάζω ὅτι οὕτως ταχέως μετατίθεσθε ἀπὸ τοῦ καλέσαντος\n\nὑμᾶς ἐν χάριτι Χριστοῦ εἰς ἕτερον εὐαγγέλιον,\n                     \n                    ὃ\n\nοὐκ ἔστιν ἄλλο· εἰ μή τινές εἰσιν οἱ ταράσσοντες ὑμᾶς καὶ\n\nθέλοντες μεταστρέψαι τὸ εὐαγγέλιον τοῦ χριστοῦ.\n                     \n     

In [25]:
# and its lemmatized version
print(AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["lemmata"][:100])

['παυλος', 'ἀπόστολος', 'ἀπ', 'ἄνθρωπος', 'ἄνθρωπος', 'ἰησοῦ', 'χριστοῦ', 'θεάομαι', 'πατήρ', 'νεκρός', 'ἐμέω', 'πᾶς', 'ἀδελφός', 'ἐκκλησιάζω', 'γαλατίας', 'χάρις', 'εἰρήνη', 'θεάομαι', 'πατήρ', 'ἡμός', 'κύριος', 'ἰησοῦ', 'χριστοῦ', 'ἁμάρτημα', 'ἡμός', 'ἐξαιρέω', 'ἡμός', 'αἰών', 'πονηρός', 'θέλημα', 'θεάομαι', 'πατήρ', 'ἡμός', 'δοκέω', 'αἰών', 'αἰών', 'ἀμάω', 'θαυμάζω', 'μετατίθημι', 'χάρις', 'χριστοῦ', 'ἕτερος', 'εὐαγγέλιον', 'τίνω', 'εἶμι', 'μεταστρέφω', 'εὐαγγέλιον', 'χριστός', 'ἄγγελος', 'οὐρανός', 'εὐαγγελίζομαι', 'Πάρος', 'εὐηγγελισάμεθα', 'ἀνάθεμα', 'προερέω', 'λέγω', 'εἶμι', 'εὐαγγελίζομαι', 'Πάρος', 'παραλαμβάνω', 'ἀνάθεμα', 'ἄρτι', 'ἄνθρωπος', 'πειθός', 'θεός', 'ζάω', 'ἄνθρωπος', 'ἀρέσκω', 'ἄνθρωπος', 'ἀρέσκω', 'χριστοῦ', 'δοῦλος', 'γνωρίζω', 'ἀδελφός', 'εὐαγγέλιον', 'εὐαγγελίζομαι', 'ὑπ', 'ἐμέω', 'ἄνθρωπος', 'ἄνθρωπος', 'παραλαμβάνω', 'διδάσκω', 'ἀποκάλυψις', 'ἰησοῦ', 'χριστοῦ', 'ἠκούσατε', 'ἐμην', 'ἀναστροφή', 'ποτός', 'ἰουδαϊσμῷ', 'ἄατος', 'ὑπερβολην', 'διώκω', 'ἐκκλησιάζω

# Global corrections

# Corrections by means of GNT

In [44]:
GNT_df = pd.read_csv("https://raw.githubusercontent.com/biblicalhumanities/Nestle1904/master/morph/Nestle1904.csv", delim_whitespace=True, index_col=False, skiprows=1, names=("book", "ch_v", "func_morph", "pos1", "pos2", "strong", "lemma", "normalized"))
GNT_df.head(5)

book ch_v func_morph   pos1   pos2 strong    lemma normalized
0  Matt  1:1     Βίβλος  N-NSF  N-NSF    976   βίβλος     Βίβλος
1  Matt  1:1   γενέσεως  N-GSF  N-GSF   1078  γένεσις   γενέσεως
2  Matt  1:1      Ἰησοῦ  N-GSM  N-GSM   2424   Ἰησοῦς      Ἰησοῦ
3  Matt  1:1    Χριστοῦ  N-GSM  N-GSM   5547  Χριστός    Χριστοῦ
4  Matt  1:1       υἱοῦ  N-GSM  N-GSM   5207     υἱός       υἱοῦ

In [50]:
len(GNT_df["normalized"].tolist())

137779

In [49]:
len(GNT_df["normalized"].unique().tolist())

18444

In [46]:
GNT_morph_lemma =[(unicodedata.normalize("NFC", morph), unicodedata.normalize("NFC", lemma)) for morph, lemma in zip(GNT_df["func_morph"].tolist(), GNT_df["lemma"].tolist())]
GNT_morph_lemma[:10]

[('Βίβλος', 'βίβλος'),
 ('γενέσεως', 'γένεσις'),
 ('Ἰησοῦ', 'Ἰησοῦς'),
 ('Χριστοῦ', 'Χριστός'),
 ('υἱοῦ', 'υἱός'),
 ('Δαυεὶδ', 'Δαυίδ'),
 ('υἱοῦ', 'υἱός'),
 ('Ἀβραάμ.', 'Ἀβραάμ'),
 ('Ἀβραὰμ', 'Ἀβραάμ'),
 ('ἐγέννησεν', 'γεννάω')]

In [51]:
len(GNT_morph_lemma)

137779

In [53]:
GNT_morph_lemma_unique = list(set(GNT_morph_lemma))
len(GNT_morph_lemma_unique)

25258

In [ ]:
GNT_morph_lemma_unique

In [28]:
NT_df = AGT[AGT["author_id"].str.startswith("tlg0031")]
NT_lemmata = []
for lemmata in NT_df["lemmata"].tolist():
    NT_lemmata.extend(lemmata)

In [40]:
NT_freqdist = nltk.FreqDist(NT_lemmata).most_common()
NT_freqdist

[('πᾶς', 1086),
 ('ἁμός', 1078),
 ('εἶπον', 828),
 ('λέγω', 786),
 ('θεάομαι', 701),
 ('θεός', 620),
 ('ἡμός', 564),
 ('ἄνθρωπος', 522),
 ('ποιέω', 487),
 ('γίγνομαι', 482),
 ('σεύω', 476),
 ('ἔχω', 465),
 ('ἰησοῦς', 454),
 ('ἔρχομαι', 421),
 ('πατήρ', 397),
 ('εἶμι', 388),
 ('ἡμέρα', 380),
 ('υἱός', 371),
 ('κύριος', 371),
 ('εἶδον', 367),
 ('πνεῦμα', 357),
 ('πολύς', 341),
 ('λόγος', 329),
 ('ἰησοῦ', 325),
 ('δίδωμι', 312),
 ('ἀδελφός', 311),
 ('οὐρανός', 273),
 ('μαθητής', 262),
 ('ἐκεῖνος', 253),
 ('οἶδα', 249),
 ('ἀκούω', 247),
 ('γῆ', 246),
 ('μέγας', 239),
 ('λαλέω', 231),
 ('πίστις', 226),
 ('ἐμέω', 219),
 ('δοκέω', 217),
 ('χριστός', 214),
 ('ἀλλ', 214),
 ('ἅγιος', 203),
 ('ἵημι', 202),
 ('νόμος', 194),
 ('γυνή', 192),
 ('γιγνώσκω', 190),
 ('χριστοῦ', 182),
 ('δύναμαι', 176),
 ('πιστεύω', 174),
 ('ἐθέλω', 170),
 ('φωνέω', 165),
 ('βασίλεια', 162),
 ('ἔθνος', 162),
 ('ἔργον', 162),
 ('πρότερος', 161),
 ('χάρις', 158),
 ('ὁράω', 157),
 ('καρδία', 156),
 ('ζωή', 151),
 ('ἀνήρ', 1

[{'i': '432951',
  'f': 'ἰησοῦς',
  'b': 'ιησους',
  'l': 'Ἰησοῦς',
  'e': 'ιησους',
  'p': 'n-p---ma-',
  'd': '15766',
  's': 'Joshua',
  'a': 'attic'},
 {'i': '432956',
  'f': 'ἰησοῦς',
  'b': 'ιησους',
  'l': 'Ἰησοῦς',
  'e': 'ιησους',
  'p': 'n-s---mn-',
  'd': '15766',
  's': 'Joshua',
  'a': 'attic epic'}]

In [ ]:
"ἰησοῦ" : "ἰησοῦς",
"ἰησοῦν" : "ἰησοῦς",
"χριστοῦ" : "χριστός",
"χριστῷ" : "χριστός",
"κύριε" : "κύριος",
"κυρίου" : "κύριος",
"ἀβρααμ" : "ἀβραάμ"

In [41]:
[(tup[0], lemma_translator(tup[0]), tup[1]) for tup in NT_freqdist[:500]]

[('πᾶς', 'all, the whole', 1086),
 ('ἁμός', 'as, when / our, ours', 1078),
 ('εἶπον', 'to speak, say', 828),
 ('λέγω',
  'to gather, pick up / to say, speak / to lay asleep, lull to sleep',
  786),
 ('θεάομαι', 'to look on, gaze at, view, behold', 701),
 ('θεός', 'God', 620),
 ('ἡμός', '', 564),
 ('ἄνθρωπος', 'manfaced', 522),
 ('ποιέω', 'to make', 487),
 ('γίγνομαι', 'gi-geno', 482),
 ('σεύω', 'to put in quick motion: to drive, hunt, chase away', 476),
 ('ἔχω', 'to have', 465),
 ('ἰησοῦς', '', 454),
 ('ἔρχομαι', 'to come', 421),
 ('πατήρ', 'a father', 397),
 ('εἶμι', 'to go', 388),
 ('ἡμέρα', 'day', 380),
 ('υἱός', 'A son', 371),
 ('κύριος', 'having power / a lord, master', 371),
 ('εἶδον', 'to see', 367),
 ('πνεῦμα', 'a blowing', 357),
 ('πολύς', 'many', 341),
 ('λόγος', 'the word', 329),
 ('ἰησοῦ', '', 325),
 ('δίδωμι', 'to give', 312),
 ('ἀδελφός', 'sons of the same mother', 311),
 ('οὐρανός', 'heaven', 273),
 ('μαθητής', 'a learner, pupil', 262),
 ('ἐκεῖνος', 'the person there, th

In [26]:
# straightforward corrections might be added here

corrections = {
    "ἔχις" : "ἔχω"
}

for key in corrections.keys():
    AGT["lemmata"] = AGT["lemmata"].apply(lambda word_list: [corrections[key] if x == key else x for x in word_list])
    AGT["lemmatized_sentences"] = AGT["lemmatized_sentences"].apply(lambda sentences_list: [[corrections[key] if x == key else x for x in sentence] for sentence in sentences_list])

In [27]:
sddk.write_file("SDAM_data/AGT/AGT_20201110.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_20201110.json"
